# Turning Raw Spotify Track Data into a Modeling Dataset  
*Integration, cleaning, and feature preparation for popularity prediction*

## 1. Introduction
Music streaming has become an ordinary part of daily life, and platforms like Spotify play a central role in how listeners discover and engage with music. The listening experience today is far more personalized than it was in the past, shaped by recommendations that adapt to individual tastes, habits, and patterns over time. At the core of this transformation is data.

While listener data is not available in this project, Spotify provides extensive information at the track and artist level. This includes audio features that describe how a song sounds, metadata that situates it in time and context, and aggregate measures of engagement such as track popularity. Together, these features offer a partial but meaningful view into how songs are represented and surfaced on the platform.

This notebook focuses on preparing and integrating Spotify track and artist datasets into a clean, modeling ready format. The resulting dataset serves as the foundation for subsequent exploratory analysis and predictive modeling of song popularity.

## 2. Libraries and Setup
This notebook uses standard Python libraries for data manipulation and preparation.

In [1]:
# Core utilities
from __future__ import annotations
# Enables postponed evaluation of type annotations for cleaner type hints and forward compatibility.

import ast
# Used to safely convert list-like strings in the raw Spotify data into actual Python lists.

# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt

# Text processing
import re


# Display settings
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)

## 3. Data Wrangling and Preparation

### 3.1 Read Files
The data preparation process begins by loading the raw Spotify track and artist datasets. These datasets are later combined and transformed to produce an enriched, track-level dataset that is saved as a parquet file for downstream exploratory analysis and modeling.

In [ ]:
tracks = pd.read_parquet("tracks.parquet")
artists = pd.read_csv("artists.csv")

### 3.2 Data Cleaning and Standardization

#### 3.2.1 Handling Missing Values
This section outlines the approach used to address missing values in the Spotify track and artist datasets. Handling missing data is an important part of preparing a reliable analytical dataset, as it helps ensure consistency and reduces the risk of introducing noise during later stages of the project.

In [ ]:
all_dfs_dict = {"tracks": tracks,
               "artists": artists}

In [ ]:
def create_null_summary(dataset_dict):
    """
    Create a dataset-level overview of size and missing values.
    """
    null_list = []
    for name, df in dataset_dict.items():
        n_rows = df.shape[0]
        n_cols = df.shape[1]
        null_sum = df.isna().any(axis=1).sum()
        total_cells = n_rows * n_cols
        # get list of column names with null values
        null_columns = df.columns[df.isnull().any()].tolist()
        # get number of columns with null values 
        n_null_columns = len(null_columns)
        # if no null columns, return an empty string
        null_columns_str = ', '.join(null_columns) if null_columns else ""
        null_list.append({
                         "dataframe":name,
                         "n_rows":n_rows,
                         "n_cols": n_cols,
                         "rows_with_nulls":null_sum,
                         "pct_rows_with_nulls": round((null_sum / n_rows * 100) if n_rows else 0, 3),
                         "columns_with_nulls":n_null_columns,
                         "null_columns_names":null_columns_str})
    return pd.DataFrame(null_list)

In [ ]:
null_df = create_null_summary(all_dfs_dict).sort_values(
    by="pct_rows_with_nulls",
    ascending=False
)
null_df

In [ ]:
# Drop nulls
tracks = tracks.dropna()
artists = artists.dropna()

# Sanity Check
tracks.isna().sum().sum(), artists.isna().sum().sum()

This initial overview provides a quick check of data completeness across both datasets. The number of missing values is extremely small relative to the overall size of the data, so those records are removed to keep the dataset clean. With missing values addressed, the next step focuses on reviewing and standardizing data types before continuing with further preparation.

#### 3.2.2 Data Type Standardization
This section focuses on reviewing and standardizing data types across the track and artist datasets. Ensuring that variables are stored using appropriate data types is a key step in data preparation, as it supports accurate transformations, comparisons, and downstream analysis. An initial inspection of each dataset is performed to confirm that columns align with their expected representations.

In [ ]:
tracks.info()

In [ ]:
tracks.head()

In [ ]:
artists.info()

In [ ]:
artists.head()

In [ ]:
dtype_specifications = {
    "float": [
        "danceability",
        "energy",
        "loudness",
        "speechiness",
        "acousticness",
        "instrumentalness",
        "liveness",
        "valence",
        "tempo",
        "followers"
    ],
    "int": [
        "popularity",
        "duration_ms",
        "key",
        "mode",
        "time_signature"
    ],
    "string": [
        "id",
        "name",
        "release_date"
    ]
}

In [ ]:
def standardize_dtypes(df, spec, errors="coerce"):
    """
    Standardize dataframe dtypes based on a spec dict.
    - Preserves missing values using pandas nullable dtypes where appropriate.
    - Ignores columns not present in the dataframe.
    """
    df = df.copy()

    # dates
    for col in spec.get("datetime", []):
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors=errors)

    # floats
    for col in spec.get("float", []):
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # integers
    for col in spec.get("int", []):
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

    # strings
    for col in spec.get("string", []):
        if col in df.columns:
            df[col] = df[col].astype("string")

    # categories
    for col in spec.get("category", []):
        if col in df.columns:
            df[col] = df[col].astype("category")

    return df

In [ ]:
artists = standardize_dtypes(artists, dtype_specifications)
tracks = standardize_dtypes(tracks, dtype_specifications)

In [ ]:
# Extract release year for modeling
tracks["release_year"] = tracks["release_date"].str.slice(0, 4).astype("Int64")

In [ ]:
# sanity check
print(artists.dtypes.sort_index())
print("-----" * 7)
print(tracks.dtypes.sort_index())

Data types were standardized where appropriate, while certain fields were intentionally left unchanged to support downstream transformations and modeling.

#### 3.2.3 Duplicate Handling
Duplicate checks are performed before parsing list-like fields to keep identifiers in a hashable format and avoid issues with row-level comparison.

In [ ]:
is_duplicate_track_id = tracks.duplicated(subset='id')
print(is_duplicate_track_id.sum())

is_duplicate_artist_id = artists.duplicated(subset='id')
print(is_duplicate_artist_id.sum())

Duplicate checks were performed using the unique identifier columns for both tracks and artists. No duplicate IDs were identified at the raw dataset level, confirming that each track and artist is uniquely represented prior to downstream joins and feature aggregation.

#### 3.2.4 Handling Outliers
This section reviews potential outliers across numeric variables using a custom function that identifies extreme values on a colum-level basis and summarizes them by proportion. This approach helps prioritize which variables warrant closer inspection.

In [ ]:
def detect_outliers_iqr(df):
    """
    Computes the proportion of IQR-based outliers in each numeric column
    to highlight variables with heavy-tailed distributions.
    """
    rows = []

    for col in df.select_dtypes(include=["number"]).columns:
        s = df[col].dropna()
        if s.empty:
            continue

        q1 = s.quantile(0.25)
        q3 = s.quantile(0.75)
        iqr = q3 - q1

        if iqr == 0:
            outlier_count = 0
        else:
            lower = q1 - 1.5 * iqr
            upper = q3 + 1.5 * iqr
            outlier_count = ((s < lower) | (s > upper)).sum()

        rows.append({
            "Column": col,
            "Outlier_%": outlier_count / len(s) * 100
        })

    return pd.DataFrame(rows)

In [ ]:
outlier_reports = {}

outlier_reports["artists"] = detect_outliers_iqr(artists)
outlier_reports["tracks"] = detect_outliers_iqr(tracks)

combined_outliers = (
    pd.concat(outlier_reports, names=["Dataset"])
      .reset_index(level=0)
      .rename(columns={"level_0": "Dataset"})
)

# Sort
combined_outliers = combined_outliers.sort_values(
    "Outlier_%",
    ascending=False
).reset_index(drop=True)

# Stylize
styled_outliers = (
    combined_outliers.style
    .format({"Outlier_%": "{:.2f}%"})
    .background_gradient(axis=0,
                         gmap = combined_outliers["Outlier_%"],
                         cmap="YlOrRd"
                        )
)

styled_outliers

In [ ]:
artists[["followers", "popularity"]].describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99])

In [ ]:
artists.sort_values("followers", ascending=False).head(10)[
    ["name", "followers", "popularity"]
]

In [ ]:
artists.sort_values("followers", ascending=True).head(10)[
    ["name", "followers", "popularity"]
]

In [ ]:
plt.hist(artists["followers"], bins=100)
plt.yscale("log")
plt.title("Artist Followers (log scale)")
plt.show()

Several audio features, such as instrumentalness, are known to exhibit highly skewed distributions due to the wide variety of musical styles represented in the dataset. These patterns are expected and do not indicate data quality issues, so no additional action was taken for those variables.

Artist-level metrics, including popularity and follower counts, were reviewed as a sanity check to ensure extreme values were plausible. Highly followed, well-known artists appeared at the upper end of the distribution, while lesser-known artists with zero or low follower counts appeared at the lower end, consistent with real-world expectations. As a result, no values were capped or removed during this step.

#### 3.2.5 Rename Columns
This section standardizes column names across datasets to reduce ambiguity and improve clarity prior to merging. Several fields share common names across tables, such as identifiers and popularity metrics, which can lead to confusion during joins and downstream analysis if left unchanged. Renaming these columns early helps make dataset roles explicit and ensures that merged features remain interpretable.

In [ ]:
tracks.rename(columns={
    "id": "track_id",
    "name": "track_name",
    "popularity": "track_popularity",
    "id_artists": "artist_id_list"
}, 
              inplace = True)

artists.rename(columns={
    "id": "artist_id",
    "name": "artist_name",
    "popularity": "artist_popularity"
}, 
              inplace = True)

After renaming overlapping columns to clearly distinguish track-level and artist-level attributes, the datasets are better structured for subsequent merges. 

#### 3.2.6 Normalizing and Parsing List-Like Fields
Some fields in the raw data represent multi-value relationships. Before expanding the data into a track–artist format and performing merges, these fields were standardized so that list-like values are consistently represented as flat Python lists. This reduces ambiguity during downstream joins and ensures that later aggregation steps behave consistently.

In [ ]:
def parse_list_like(x):
    """
    Convert stringified Python lists into real lists.
    Returns the original value if it is already a list or cannot be parsed.
    """
    if isinstance(x, list):
        return x
    if pd.isna(x):
        return x
    if isinstance(x, str) and x.startswith("[") and x.endswith("]"):
        try:
            val = ast.literal_eval(x)
            return val if isinstance(val, list) else x
        except Exception:
            return x
    return x

In [ ]:
tracks["artists"] = tracks["artists"].apply(parse_list_like)
tracks["artist_id_list"] = tracks["artist_id_list"].apply(parse_list_like)
artists["genres"] = artists["genres"].apply(parse_list_like)


In [ ]:
def normalize_artist_id_list(artist_id_list):
    """
    Normalize artist_id_list into a flat Python list of clean artist ID strings.
    Safe against NaN, strings, lists, and numpy arrays.
    """

    # Handle None
    if artist_id_list is None:
        return []

    # Handle scalar NaN 
    if isinstance(artist_id_list, (float, np.floating)) and pd.isna(artist_id_list):
        return []

    # If numpy array, convert to list
    if isinstance(artist_id_list, np.ndarray):
        artist_id_list = artist_id_list.tolist()

    # String: parse list-like strings or wrap single value
    if isinstance(artist_id_list, str):
        s = artist_id_list.strip()
        if s.startswith("[") and s.endswith("]"):
            try:
                artist_id_list = ast.literal_eval(s)
            except Exception:
                return [s.strip().strip("'\"")]
        else:
            return [s.strip().strip("'\"")]

    # Wrap non-list scalars
    if not isinstance(artist_id_list, list):
        return [str(artist_id_list).strip().strip("'\"")]

    # Flatten one level if list-of-lists
    flattened = []
    for item in artist_id_list:
        if isinstance(item, list):
            flattened.extend(item)
        else:
            flattened.append(item)

    # Clean 
    cleaned = []
    for item in flattened:
        if item is None:
            continue
        if isinstance(item, (float, np.floating)) and pd.isna(item):
            continue
        item_str = str(item).strip().strip("'\"")
        if item_str:
            cleaned.append(item_str)

    return cleaned

In [ ]:
tracks = tracks.copy()

tracks["artist_id_list"] = tracks["artist_id_list"].apply(normalize_artist_id_list)


After parsing, artist and genre information is explicitly represented as structured lists rather than raw strings. This makes multi-artist relationships transparent and prepares the data for subsequent aggregation without altering the underlying unit of analysis.

### 3.3 Dataset Integration 
Some tracks in the dataset are associated with multiple artists, while artist-level information is stored separately. To incorporate artist information without altering the original tracks dataset, the data is temporarily reshaped into a track–artist format, enriched with artist metadata, and then summarized back to the track level. The result is a new dataset that combines track features with aggregated artist information for downstream modeling.

#### 3.3.1 Expanding Tracks into a Long Track-Artist Table
Some tracks are associated with multiple artists. In this step, the data is expanded so that each row represents a single track–artist relationship. This intermediate table allows artist-level information to be joined correctly.

In [ ]:
track_artist_long = (
    tracks[["track_id", "artist_id_list"]]
    .explode("artist_id_list", ignore_index=True)
    .rename(columns={"artist_id_list": "artist_id"})
)

# Composite key 
track_artist_long["track_artist_id"] = (
    track_artist_long["track_id"].astype(str) + "_" +
    track_artist_long["artist_id"].astype(str)
)

#### 3.3.2 Adding Artist Information to the Long Track-Artist Table
Artist-level attributes such as popularity, follower counts, genres, and artist names are merged into the track–artist table created in the previous step.

In [ ]:
# merge tracks with artist information 
track_artist_long = track_artist_long.merge(
    artists[["artist_id","artist_name","genres", "artist_popularity", "followers"]],
    on="artist_id",
    how="left"
)
track_artist_long["n_genres"] = track_artist_long["genres"].apply(
    lambda g: len(g) if isinstance(g, list) else 0
)
track_artist_long.head()

#### 3.3.3 Creating Track-Level Artist Features
Artist-level information is then summarized back to the track level. This includes features such as the number of artists on a track and aggregated artist popularity and follower metrics.

In [ ]:
artist_agg = (
    track_artist_long
    .groupby("track_id", as_index=False)
    .agg(
        artist_popularity_mean=("artist_popularity", "mean"),
        artist_popularity_max=("artist_popularity", "max"),
        artist_followers_mean=("followers", "mean"),
        artist_followers_max=("followers", "max"),
        n_artists=("artist_id", "nunique"), 
        artist_genres_mean = ("n_genres", "mean"),
        artist_genres_max = ("n_genres", "max"),
    )
)

In [ ]:
artist_agg.head()

#### 3.3.4 Creating an Enriched Tracks Dataset
The aggregated artist features are joined to the original track data to form a new, enriched tracks dataset, while preserving the original tracks table for independent analysis.

In [ ]:
tracks_enriched = tracks.merge(
    artist_agg,
    on="track_id",
    how="left"
)

tracks_enriched.head()

In [ ]:
tracks_enriched.isna().mean().sort_values(ascending=False)

In [ ]:
artist_agg_cols = [
    "artist_followers_mean",
    "artist_followers_max",
    "artist_popularity_mean",
    "artist_popularity_max"
]

tracks_enriched_model = tracks_enriched.dropna(subset=artist_agg_cols)

In [ ]:
tracks_enriched_model.isna().mean().sort_values(ascending=False)

In [ ]:
output_path = "tracks_enriched_model.parquet"

tracks_enriched_model.to_parquet(output_path, index=False)

print(f"Enriched dataset successfully saved to '{output_path}'.")
print(f"Final shape: {tracks_enriched_model.shape}")

A small number of tracks lacked aggregated artist metadata. Because this represented a minimal portion of the data and was limited to artist-derived features, these rows were excluded from the modeling dataset.

## 4. Data Preparation Summary and Key Takeaways
**Summary**

This notebook focused on cleaning and preparing Spotify track and artist data for analysis. The raw data included track-level audio features and artist-level metadata, with some tracks linked to multiple artists.

To handle this, tracks were temporarily expanded to a track–artist format so artist information could be added correctly. Artist features were then summarized back to the track level to create a new enriched dataset.

**Key Takeaways**
* Track and artist data exist at different levels and required reshaping to be combined correctly.
* Tracks with multiple artists were handled by expanding to a track-artist format and then aggregating artist features.
* List-like fields were normalized to avoid issues during merging and aggregation.
* Release year was used instead of full dates to avoid adding false precision.
* Missing values were minimal in the enriched tracks dataset. About 2% of rows were removed for modeling.
* The final enriched dataset is ready for further analysis in a separate notebook.
